In [0]:
# Define the path to your Parquet files
# parquet_file_path1 = "dbfs:/Silver/2024/month=03/date=2024-03-15/df1_20240315_132137.parquet"
parquet_file_path1 = "dbfs:/Silver/2_wheelers/202403/29/*.parquet"
parquet_file_path2 = "dbfs:/Silver/4_wheelers/202403/29/*.parquet"

# Read the Parquet file into a DataFrame
# df1 = spark.read.parquet(parquet_file_path1)
# display(df1)
# df2= spark.read.parquet(parquet_file_path2)
# display(df2)

In [0]:
#nnnn
from pyspark.sql.functions import col, input_file_name, lit, when, split, regexp_replace
import pyspark.sql.functions as F

# stg_df_1 = spark.read.format('parquet').option('Header', 'True').option('inferSchema', 'True').load(parquet_file_path1) \
#     .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(F.col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(F.col('inputfilename') == '2-WheelSales', lit('T')).otherwise(lit('F'))).select(F.col('brand').alias('BrandName'), F.col('WheelerKey'))


combined_stg =  spark.read.format('parquet').option('Header', 'True').option('inferSchema', 'True').load(parquet_file_path2) \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(F.col('inputfilename') == '4-WheelSales', lit('T')).otherwise(lit('F'))).select(col('transmission').alias('Transmission'), 'wheelerkey')

# combined_stg = stg_df_1.unionByName(stg_df_2)
print("combined_count :", combined_stg.count())

print("combined_distinct :" ,combined_stg.count())
combined_stg.show()

dim_df = spark.sql("""SELECT * FROM  dev.dimtransmission""")
print('dim_df show...')
dim_df.show()
print("check for new records...")

new_records = combined_stg.join(dim_df, 'Transmission', 'left_anti')
new_records.show()

updated_records = combined_stg.join(dim_df, combined_stg['Transmission'] == dim_df['Transmission'], 'inner').select(dim_df['*'])

print("matching from src to target ")
updated_records.show()

combined_records = new_records.unionByName(updated_records.drop('pkTransmissionId'))

print('combined....')
combined_records = combined_records.dropDuplicates(['Transmission', 'WheelerKey'])
print('combined_records df count:', combined_records.count())
combined_records.show()

combined_records.write.mode('overwrite').saveAsTable("dev.dimtransmission")

print("success") 

combined_count : 91440
combined_distinct : 91440
+------------+----------+
|Transmission|wheelerkey|
+------------+----------+
|   Automatic|         F|
|   Automatic|         F|
|      Manual|         F|
|   Automatic|         F|
|   Automatic|         F|
|   Automatic|         F|
|   Automatic|         F|
|      Manual|         F|
|   Automatic|         F|
|   Automatic|         F|
|      Manual|         F|
|   Automatic|         F|
|   Automatic|         F|
|   Automatic|         F|
|   Automatic|         F|
|      Manual|         F|
|   Automatic|         F|
|   Automatic|         F|
|      Manual|         F|
|   Automatic|         F|
+------------+----------+
only showing top 20 rows

dim_df show...
+----------------+------------+----------+
|pkTransmissionId|Transmission|WheelerKey|
+----------------+------------+----------+
+----------------+------------+----------+

check for new records...
+------------+----------+
|Transmission|wheelerkey|
+------------+----------+
|   Automa

In [0]:
%sql
select * from dev.dimtransmission

pkTransmissionId,Transmission,WheelerKey
1,Automatic,F
2,Manual,F


In [0]:
%sql
truncate table dev.dimtransmission

In [0]:
# distinct_transmission = df2.select("transmission").distinct()
# distinct_transmission.show()

+------------+
|transmission|
+------------+
|   Automatic|
|      Manual|
+------------+



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime 

def transmission_scd1(path):
    stg_df = spark.read.format('parquet') \
                    .option('Header', 'True') \
                    .load(path) \
                    .withColumn('inputfilename', input_file_name()) \
                    .withColumn('inputfilename',  concat_ws("/", expr("slice(split(inputfilename, '/'), 1, 3)"))) \
                    .withColumn('wheelerkey', when(col('inputfilename') == 'dbfs:/Silver/4_wheelers', lit('F')).otherwise(lit('T'))) \
                    .select(col('transmission').alias('Transmission'), 'wheelerkey').distinct()

    print("read the table")

    dim_df = spark.sql("select * from dev.dimtransmission")

    # dim_df.show()
    print("check for new and updated records...")
    new_records = stg_df.join(dim_df, 'Transmission', 'left_anti')
    new_records.show()

    print("check for matched records")

    matched_records = stg_df.join(dim_df,  stg_df['Transmission']==dim_df['Transmission'], 'inner').select(dim_df['*'])

    matched_records.show()

    print("final_df")

    final_df = matched_records.drop('pkTransmissionId').unionByName(new_records)

    final_df.show()

    final_df.write.mode('overwrite').saveAsTable('dev.dimtransmission')

    print('success')


    

In [0]:
transmission_scd1(parquet_file_path2)

read the table
check for new and updated records...
+------------+----------+
|Transmission|wheelerkey|
+------------+----------+
|   Automatic|         F|
|      Manual|         F|
+------------+----------+

check for matched records
+----------------+------------+----------+
|pkTransmissionId|Transmission|WheelerKey|
+----------------+------------+----------+
+----------------+------------+----------+

final_df
+------------+----------+
|Transmission|WheelerKey|
+------------+----------+
|   Automatic|         F|
|      Manual|         F|
+------------+----------+

success


In [0]:
%sql
select * from dev.dimtransmission

pkTransmissionId,Transmission,WheelerKey
1,Automatic,F
2,Manual,F


In [0]:
# from delta.tables import *
# from pyspark.sql.functions import lit
# delta_df=DeltaTable.forPath(spark, "dbfs:/delta_tables/Dim6")

# delta_df.alias('target').merge(
#     source=df2.select('transmission').distinct().alias('source'),
#     condition='target.Transmission=source.transmission'
# ).whenMatchedUpdate(
#     set={
#         'WheelerKey': lit('F')
#     }
# ).whenNotMatchedInsert(
#     values={
#         'Transmission': 'source.transmission',
#         'WheelerKey': lit('F')
#     }
# ).execute()